## Dependências

Instalação no jupyter:

```
import sys
!{sys.executable} -m pip install requests bs4 selenium
```

In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time

In [7]:
link = "https://www.tripadvisor.com.br/Attractions-g2342964-Activities-Crato_State_of_Ceara.html"
#link = "https://www.tripadvisor.com.br/Attractions-g304560-Activities-Recife_State_of_Pernambuco.html"
#https://www.tripadvisor.com.br/Attractions-g303428-Activities-oa60-Joao_Pessoa_State_of_Paraiba.html
#https://www.tripadvisor.com.br/Attractions-g303428-Activities-oa120-Joao_Pessoa_State_of_Paraiba.html#FILTERED_LIST
driver = webdriver.Safari() # mude pro seu navegador favorito
driver.get(link)
time.sleep(1)

In [8]:
buttons = driver.find_elements_by_class_name("attractions-attraction-overview-main-TopPOIs__see_more--2Vsb-")
for button in buttons:
    if button.is_displayed():
        driver.execute_script("arguments[0].click();", button)

soup = BeautifulSoup(driver.page_source, 'html.parser')
info = soup.find_all("div", class_ = "attractions-attraction-overview-pois-PoiInfo__info--239IR")

names = []
ref = []
categories = []
reviews = []

for attraction in info:
    nl = attraction.find("a", class_ = "attractions-attraction-overview-pois-PoiInfo__name--SJ0a4")
    names.append(nl.text)
    ref.append(nl.get("href"))
    categories.append(attraction.find("div", class_ = "attractions-attraction-overview-pois-PoiInfo__geo_category_container--rcvCM").text)
    try:
        reviews.append(attraction.find("span", class_ = "reviewCount styleguide-bubble-rating-BubbleRatingWithReviewCount__reviewCount--37tMc").text.split()[0])
    except:
        reviews.append("N/A")
        
driver.quit()

In [9]:
#por algum motivo as outras paginas são diferentes da primeira
oa = 30
if(len(names) == 30):
    while True:
        req = get("https://www.tripadvisor.com.br/Attractions-g303428-Activities-oa" + str(oa) + "-Crato_State_of_Ceara.html")
        soup = BeautifulSoup(req.content, 'html.parser')
        info = soup.find_all("div", class_ = "flexible")
        for attraction in info:
            nl = attraction.find("div", class_ = "tracking_attraction_title listing_title").find("a")
            names.append(nl.text)
            ref.append(nl.get("href"))
            try: # eu queria saber lancar excecoes decentemente
                categories.append(attraction.find("span", class_ = "matchedTag noTagImg").text)
            except:
                categories.append("N/A")
            try:
                reviews.append(attraction.find("span", class_ = "more").text.split()[0])
            except:
                reviews.append("N/A")
        if(len(info) < 30):
            break
        else:
            oa += 30

In [10]:
#abrir a pagina especifica da atracao

address = []
days = []
hours = []
ratings = []

# distribuicao das avaliacoes
exc = []
vg = []
avg = []
poor = []
ter = []

certified = []

i = 1
t = len(ref)
for page in ref:
    d, r = False, False
    req = get("https://www.tripadvisor.com.br" + page)
    print("(" + str(i) + "/" + str(t) + ") Current page: " + page)
    soup = BeautifulSoup(req.content, 'html.parser')
    
    try:
        address.append(soup.find("span", class_ = "textAlignWrapper address").text)
    except:
        address.append("N/A")
        
    try:
        days.append(soup.find("div", class_ = "dayRange ui_column is-4").text)
        d = True
        hours.append(soup.find("div", class_ = "timeRange").text)
    except:
        hours.append("N/A")
        if d is False:
            days.append("N/A")
            
    try:
        ratings.append(soup.find("span", class_ = "overallRating").text)
        r = True
        dist = soup.find_all("span", class_ = "row_count row_cell")
    except:
        ratings.append("N/A")
        if r is False:
            exc.append("N/A")
            vg.append("N/A")
            avg.append("N/A")
            poor.append("N/A")
            ter.append("N/A")
            
    if r is True:
        exc.append(dist[0].text)
        vg.append(dist[1].text)
        avg.append(dist[2].text)
        poor.append(dist[3].text)
        ter.append(dist[4].text)
    
    try:
        soup.find("div", class_ = "attractions-attraction-detail-about-card-Award__award_text--1OfPx").text
        certified.append(True)
    except:
        certified.append(False)
        
    i += 1
print("Done")

(1/11) Current page: /Attraction_Review-g2342964-d10068287-Reviews-Araripe_Plateau-Crato_State_of_Ceara.html
(2/11) Current page: /Attraction_Review-g2342964-d4550304-Reviews-Geopark_Araripe-Crato_State_of_Ceara.html
(3/11) Current page: /Attraction_Review-g2342964-d4600991-Reviews-Praca_da_Se-Crato_State_of_Ceara.html
(4/11) Current page: /Attraction_Review-g2342964-d4376259-Reviews-Museu_Historico_do_Crato-Crato_State_of_Ceara.html
(5/11) Current page: /Attraction_Review-g2342964-d15075963-Reviews-Centro_de_Eventos_Do_Cariri-Crato_State_of_Ceara.html
(6/11) Current page: /Attraction_Review-g2342964-d4376252-Reviews-Fosseis_DNPM_Museum-Crato_State_of_Ceara.html
(7/11) Current page: /Attraction_Review-g2342964-d4376253-Reviews-Museu_de_Historia_Natural_da_Universidade_Regional_do_Cariri_URCA-Crato_State_of.html
(8/11) Current page: /Attraction_Review-g2342964-d10063989-Reviews-Museu_Benedito_Jose_Teles-Crato_State_of_Ceara.html
(9/11) Current page: /Attraction_Review-g2342964-d4126941-

In [11]:
# Cria o dataframe e o exporta (deixar o encoding como cp1252 para permitir acentos)
df = pd.DataFrame({'Ref': ref, 'Name': names, 'Category': categories, "Address": address, "Days": days, "Hours": hours, 'Number of Reviews': reviews, 'Rating': ratings, 'Excellent': exc, 'Very Good': vg, 'Average': avg, 'Poor': poor, 'Terrible': ter, 'Certified': certified})

df
df.to_csv('crato.csv', index = False, encoding = 'cp1252')

## Obtendo os nomes e endereços das cidades

Apenas é necessario o código de uma cidade para acessar a página de atrações desta. Mesmo estando a parte do "cont" errada, será feito um redirecionamento para a página correta com base no código. O código abaixo irá obter todas as páginas que estejam no range (caso existam).

In [8]:
base = "https://www.tripadvisor.com.br/Attractions-g"
cont = "-Activities-Campo_Grande_State_of_Mato_Grosso_do_Sul.html"

urls = []
cities = []
for code in range(303400, 303500):
    req = get(base + str(code) + cont)
    soup = BeautifulSoup(req.content, 'html.parser')
    if soup.find("div", class_ = "attractions-attraction-overview-main-PoiHeader__title--3Mpd9"):
        cities.append(soup.find("div", class_ = "attractions-attraction-overview-main-PoiHeader__title--3Mpd9").text.partition(":")[0])
        urls.append(req.url)


In [37]:
try:
    df2 = pd.read_csv('links.csv', delimiter=",", encoding = 'cp1252') #abrindo o arquivo
#df2 = pd.DataFrame({'City': cities, 'Url': urls})
except:
    df2 = None


In [38]:
df4 = pd.concat([df2, pd.DataFrame({'City': cities, 'Url': urls})]).drop_duplicates()
df4.to_csv('links.csv', index = False, encoding = 'cp1252')

df4

,City,Url
0,Uberlândia,https://www.tripadvisor.com.br/Attractions-g30...
1,Varginha,https://www.tripadvisor.com.br/Attractions-g30...
2,São Paulo,https://www.tripadvisor.com.br/Attractions-g30...
3,Belém,https://www.tripadvisor.com.br/Attractions-g30...
4,Belo Horizonte,https://www.tripadvisor.com.br/Attractions-g30...
5,Campinas,https://www.tripadvisor.com.br/Attractions-g30...
6,Diamantina,https://www.tripadvisor.com.br/Attractions-g30...
7,Fortaleza,https://www.tripadvisor.com.br/Attractions-g30...
